#### Import Modules

In [4]:
import requests
import pandas
import json
import geopandas
import matplotlib.pyplot as plt
import urllib
import shutil
import zipfile
from shapely.geometry import Point
import os

### Download Two Datatsets

#### Download Park and Rides & Transit Hubs from MN Geospatial Commons

In [5]:
#Shapefile URl
mndataurl = r'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_metc/trans_park_rides_transit_centers/shp_trans_park_rides_transit_centers.zip'
#Path to Save Shapefile
mnpath = r'C:\Users\gregkohler1\Documents\GIS5571_Files\Lab1\Lab1Data'
#Name of Shapefile
mnoutput = r'C:\Users\gregkohler1\Documents\GIS5571_Files\Lab1\Lab1Data\MNTransitHubs.zip'
#Get Shapefile from URL
mnresponse = requests.get(mndataurl, stream=True)
#Writes Shapefile to Local Disk
if mnresponse.status_code == 200:
    with open(mnoutput, 'wb') as file:
        shutil.copyfileobj(mnresponse.raw, file) #
#Extract Zip with Shapefiles
    with zipfile.ZipFile(mnoutput, 'r') as zip_MN:
        zip_MN.extractall(mnpath)
else:
    print("Wrong Status Code")

#### Download Bus Garages from Google Places

In [7]:
#URL for API
googleurl = 'https://maps.googleapis.com/maps/api/place/textsearch/json'
#Defines query for bus garages in the Twin Cities
garagequery = {
    'query': "Bus Garage in Twin Cities",
    'key': 'API-KEY' 
}

#Pull Data from Url
googler = requests.get(googleurl, params=garagequery)

#Format Data as JSON, Check data status
if googler.status_code == 200:
    googledata = googler.json()
    #Create Blank List to Write JSON Items to
    garages = []
    #Pulls values from JSON to make geometry for shapefile
    for gplace in googledata['results']:
        name = gplace['name']
        latitude = gplace['geometry']['location']['lat']
        longitude = gplace['geometry']['location']['lng']
        garages.append({'name': name, 'geometry': Point(longitude, latitude)})
else:
    print("Wrong Status Code")

        #Create geodataframe and writes shapefile to local disk
googledataframe = geopandas.GeoDataFrame(garages, geometry='geometry', crs ='WGS84')
googledataframe.to_file(r'C:\Users\gregkohler1\Documents\GIS5571_Files\Lab1\Lab1Data\BusGarages.shp')

C:\Users\gregkohler1\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


#### Convert to Same Coordinate Reference System

In [8]:
# Opens Up Both Datasets
garagesgdf = geopandas.read_file(r'C:\Users\gregkohler1\Documents\GIS5571_Files\Lab1\Lab1Data\BusGarages.shp')
parkridegdf = geopandas.read_file(r'C:\Users\gregkohler1\Documents\GIS5571_Files\Lab1\Lab1Data\ParkAndRideLotsTransitCenters.shp')
#Converts Both Datasets to Same CRS
garagesgdf = garagesgdf.to_crs("4326")
parkridegdf = parkridegdf.to_crs("4326")
#Saves Both Datasets to New Name with New CRS
garagesgdf.to_file(r'C:\Users\gregkohler1\Documents\GIS5571_Files\Lab1\Lab1Data\BusGarages_NewCRS.shp')

C:\Users\gregkohler1\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


#### Clean Up Park & Ride Table

In [9]:
#Removes Unwanted Columns from parkandride GeoDataFrame
deletelist = ['TCode', "PnRNum", 'LocDetails', 'YearClose', 'LandOwnr', 'ShrUseDesc','PrkngType', 'CurrentCpc', 'PlanCapc', 'Mpls_Corr', 'Stp_Corr', 'Shelter', 'Bench', 'IndoorWt', 'TrashRec', 'DropOff','BikeRacks', 'BikeLocker', 'RT_Sign', 'Notes', 'created_us', 'created_da', 'last_edite', 'last_edi_1', 'ATIS_ID']
parkridegdf = parkridegdf.drop(columns=deletelist, axis=1)
#Saves Cleaned Up GeoDataFrame to Shapefile
parkridegdf.to_file(r'C:\Users\gregkohler1\Documents\GIS5571_Files\Lab1\Lab1Data\ParkAndRideLotsTransitCenters_CRSandColumns.shp')

C:\Users\gregkohler1\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


#### Join Bus Garages and Park and Rides and saves to Lab1 Geodatabase

In [10]:
#Perform Spatial Join
arcpy.analysis.SpatialJoin(
    target_features=r'C:\Users\gregkohler1\Documents\GIS5571_Files\Lab1\Lab1Data\ParkAndRideLotsTransitCenters_CRSandColumns.shp',
    join_features=r'C:\Users\gregkohler1\Documents\GIS5571_Files\Lab1\Lab1Data\BusGarages_NewCRS.shp',
    out_feature_class=r"C:\Users\gregkohler1\Documents\GIS5571_Files\Lab1\Lab1.gdb\ParkRideBusGarages_Join",
    join_operation="JOIN_ONE_TO_MANY",
    join_type="KEEP_COMMON",
    match_option="WITHIN_A_DISTANCE",
    search_radius="5 Kilometers",
    distance_field_name=""
)

<Result 'C:\\Users\\gregkohler1\\Documents\\GIS5571_Files\\Lab1\\Lab1.gdb\\ParkRideBusGarages_Join'>

#### Rename Joined Field to Bus Garage within 5km

In [11]:
#Edits Name of New Joined Field to be Bus Garages Within 5km of Park and Rides
edit = arcpy.da.Editor(arcpy.env.workspace)                                 
arcpy.AlterField_management(in_table=r'C:\Users\gregkohler1\Documents\GIS5571_Files\Lab1\Lab1.gdb\ParkRideBusGarages_Join',
                            field='name_1',
                            new_field_name='BusGaragewithin5km',
                            new_field_alias='Bus Garage Within 5km')

<Result 'C:\\Users\\gregkohler1\\Documents\\GIS5571_Files\\Lab1\\Lab1.gdb\\ParkRideBusGarages_Join'>

#### Write Out Table

In [12]:
#Set feature class
joinpath = "ParkRideBusGarages_Join"
# Create search cursor to go through the table and pull out fields and data.
fields = [field.name for field in arcpy.ListFields(joinpath)]
data = [row for row in arcpy.da.SearchCursor(joinpath, fields)]
# Convert data to dataframe
joindf = pandas.DataFrame(data, columns=fields)
# Print table
joindf

,OBJECTID,Shape,Join_Count,TARGET_FID,JOIN_FID,Name,FcltyType,Status,Address,City,Provider,YearOpen,FcltyOwnr,SharedUse,x_coord,y_coord,BusGaragewithin5km
0,1,"(-93.2310461697449, 44.85567435019118)",1,1,1,30th Ave Station Park & Ride,PR,Open,8101 Winstead Way,Bloomington,Metro Transit Rail,2004.0,Metropolitan Council,N,-93.231054,44.855682,Metro Transit South Garage
1,2,"(-93.22963590956954, 44.934764137554964)",1,2,14,38th Street Station Transit Center,TC,Open,2902 38th St E,Minneapolis,Metro Transit,2004.0,Metropolitan Council,N,-93.229644,44.934772,Greyhound: Bus Stop
2,3,"(-93.33109257905926, 45.07305937628922)",1,5,9,65th Ave & Brooklyn Blvd,PR,Open,6503 Brooklyn Blvd,Brooklyn Center,Metro Transit,1995.0,Metropolitan Council,N,-93.331100,45.073067,Metro Transit Martin J Ruter Garage
3,4,"(-93.28735759557657, 45.08839555111382)",1,6,9,Hwy 252 & 73rd Ave N,PR,Inactive,7317 West River Rd,Brooklyn Park,Metro Transit,1991.0,Metropolitan Council,N,-93.287365,45.088404,Metro Transit Martin J Ruter Garage
4,5,"(-93.02439401896879, 44.996880027990926)",1,8,17,Aldrich Arena,PR,Closed,1850 White Bear Ave,Maplewood,,2002.0,,,-93.024402,44.996888,ISD 622 Transportation
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,112,"(-93.2980565250885, 44.950244096887616)",1,174,13,Uptown Transit Station,TC,Open,2855 Hennepin Ave S,Minneapolis,Metro Transit,2001.0,Metropolitan Council,N,-93.298064,44.950252,Metro Transit North Loop Garage
112,113,"(-93.2980565250885, 44.950244096887616)",1,174,16,Uptown Transit Station,TC,Open,2855 Hennepin Ave S,Minneapolis,Metro Transit,2001.0,Metropolitan Council,N,-93.298064,44.950252,Fourth Street North Transit Station
113,114,"(-93.2980565250885, 44.950244096887616)",1,174,18,Uptown Transit Station,TC,Open,2855 Hennepin Ave S,Minneapolis,Metro Transit,2001.0,Metropolitan Council,N,-93.298064,44.950252,Marquette Av S At 5 St S Sw
114,115,"(-93.04089733484409, 45.01398056385199)",1,189,17,Hwy 36 Station,PR,Future,,Maplewood,Metro Transit,0.0,,N,0.000000,0.000000,ISD 622 Transportation
